In [13]:
# Settings

# General settings
data_folder = "/home/andrea/CIMA/PROJECT_Volta/data/integrated_data/" #"/home/andrea/Desktop/Test/series/" 
start_analysis = "2017-01-01 00:00"
end_analysis = "2020-12-31 23:00"
freq="D"

# Hmc settings
domain = "volta"
hmc_output = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/hmc.hydrograph_infMod2.txt"
hmc_static_gridded = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/data_static/gridded/"
hmc_static_point = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/data_static/point/"

# Data
et_series = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/average_ET_old.txt"
rainfall_series = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/imerg/average_rainfall_mm_h.txt"

In [26]:
# Imports
%matplotlib inline

from ipywidgets import interactive,HBox
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from pysheds.grid import Grid

import ipywidgets as widgets
from IPython.display import display, HTML
import os, glob
import datetime as dt
import matplotlib.lines as mlines

# Define custom functions
def create_df(choices):
    df = pd.DataFrame(index=pd.date_range(start_time,end_time,freq=freq), columns=choices)
    for name in choices:
        series = pd.read_csv(os.path.join(data_folder, name + ".csv"), index_col=0, header=0, parse_dates=True)
        df[name] = series.reindex(pd.date_range(start_time,end_time,freq=freq), method=None)
    return df

def multiplot(widg):
    choices = widg['new']
    df = create_df(choices)
    data = df.loc[:, choices] if choices else df
    output.clear_output(wait=True)
    with output:
        ax = data.plot(figsize=(10,7))
        plt.show()
        
def combinedplot(widg):
    choices = widg['new']
    data_et = series["ET"].loc[:, choices] if choices else series
    data_rain = series["rain"].loc[:, choices] if choices else series
    data_qMod = series["discharge_mod"].loc[:, choices] if choices else series
    data_qObs = series["discharge_obs"].loc[:, choices] if choices else series
    output2.clear_output(wait=True)
    with output2:
        ax = data_rain.plot(figsize=(15,7), color='c') 
        plt.ylim(bottom=0)
        plt.ylabel("mm of rain")
        axx = ax.twinx()
        #ax1 = data_et.plot(figsize=(15,7), color='g')
        ax2 = data_qMod.plot(figsize=(15,7), color='b')
        ax3 = data_qObs.plot(figsize=(15,7), color='r', style='.')
        plt.ylim(bottom=0)
        plt.ylabel("mm")
        cyan_line = mlines.Line2D([], [], color='cyan', label='rain')
        green_line = mlines.Line2D([], [], color='green', label='et')
        blue_line = mlines.Line2D([], [], color='blue', label='mod_dis')
        red_line = mlines.Line2D([], [], color='red', label='obs_dis')
        
        plt.legend(handles=[cyan_line, green_line, blue_line, red_line])
        plt.show()

def read_discharge_hmc(file='', col_names=None):
    custom_date_parser = lambda x: dt.datetime.strptime(x, "%Y%m%d%H%M")
    hmc_discharge_df = pd.read_csv(file, header=None, delimiter=r"\s+", parse_dates=[0], index_col=[0], date_parser=custom_date_parser)
    hmc_discharge_df.columns = col_names
    return hmc_discharge_df

In [15]:
# Read static inputs

# Sections
sections = tabular = pd.read_csv(os.path.join(hmc_static_point, domain + ".info_section.txt"), sep="\s+", header=None)
rHMC, cHMC, basin_name, section_name = tabular.values[:,0], tabular.values[:,1], tabular.values[:,2], tabular.values[:,3]

# Pointers
grid = Grid.from_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"))
pnt = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"), dtype=np.int8)
areacell = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".areacell.txt"))

In [16]:
# Compute basin cell area
basin_area = pd.DataFrame(index=section_name, columns=["Area (m2)"])
dirmap_HMC = (8, 9, 6, 3, 2, 1, 4, 7)

for ix, iy, basin, name in zip(cHMC, rHMC, basin_name, section_name):
        basin = grid.catchment(fdir=pnt, x=ix-1, y=iy-1, dirmap=dirmap_HMC, xytype='index')
        mask = np.where(basin>0, 1, np.nan)
        basin_area.loc[name, "Area (m2)"] = np.nansum(mask*areacell).astype("float32")

In [17]:
# Read hmc output
mod_out = read_discharge_hmc(hmc_output, section_name)

In [18]:
# Set timing
start_time = dt.datetime.strptime(start_analysis, "%Y-%m-%d %H:%M")
end_time = dt.datetime.strptime(end_analysis, "%Y-%m-%d %H:%M")

In [19]:
# Read datasets
series = {}

series["ET"] = pd.read_csv(et_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
series["rain"] = pd.read_csv(rainfall_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
dis_out = mod_out[start_time:end_time]

missing_data = [i for i in section_name if not os.path.isfile(os.path.join(data_folder, i + ".csv"))]
display("WARNING! Data for sections " + ", ".join(missing_data) + " are missing!")
dis_in = create_df([i for i in section_name if i not in missing_data])

'WARNING! Data for sections Bayé, Gorée, Pletou, Rte-SokodeBassar are missing!'

In [20]:
# Convert all series to mm/day
series["rain"] = series["rain"].resample("D").sum()
series["discharge_mod"] = pd.DataFrame(index = series["rain"].index, columns = dis_out.columns)
series["discharge_obs"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)
series["availability"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)

for name in dis_out.columns:
    series["discharge_mod"][name] = (dis_out[name] * (1000*3600)/float(basin_area.loc[name, "Area (m2)"])).resample("D").sum()

for name in series["discharge_obs"].columns:
    series["discharge_obs"][name] = (dis_in[name] * (1000*3600*24)/float(basin_area.loc[name, "Area (m2)"]))
    series["availability"][name] = dis_in[name].resample("D").count()

In [21]:
series_annual = {}

for stype in series.keys():
    series_annual[stype] = series[stype].resample("Y").sum()
    index = pd.Index([str(y) + "_" + stype for y in np.unique(series_annual[stype].index.year)])
    series_annual[stype].set_index(index, inplace=True)

In [22]:
out_table = pd.concat([series_annual[i].T for i in series_annual.keys()], axis=1)
out_table = out_table.reindex(sorted(out_table.columns), axis=1)

display(out_table)
out_table.to_csv("/home/andrea/Desktop/summary_balance_old.csv")

,2017_ET,2017_availability,2017_discharge_mod,2017_discharge_obs,2017_rain,2018_ET,2018_availability,2018_discharge_mod,2018_discharge_obs,2018_rain,2019_ET,2019_availability,2019_discharge_mod,2019_discharge_obs,2019_rain,2020_ET,2020_availability,2020_discharge_mod,2020_discharge_obs,2020_rain
Arly_doudouro,1021.716237,0.0,56.482268,0.000000,845.820500,894.075218,0.0,49.021629,0.000000,790.921938,1037.032913,0.0,112.273714,0.000000,970.648474,932.353303,0.0,153.002817,0.000000,956.784302
Atchangbade,1054.923667,0.0,204.208195,0.000000,1245.472651,938.838078,0.0,195.998395,0.000000,1160.264104,920.578267,0.0,236.333324,0.000000,1226.638043,1078.610595,0.0,421.526804,0.000000,1469.368808
Badara,1035.307004,308.0,11.058509,31.662802,839.624407,1006.291006,290.0,257.269213,79.559732,1242.660414,1218.232212,299.0,309.603239,49.561959,1314.401025,1068.128033,312.0,211.441192,66.444979,1082.414506
Bagre_aval_au_pont,930.260185,258.0,7.215281,0.679877,767.246833,862.718390,287.0,85.264178,2.948007,899.876780,841.603791,365.0,55.959032,0.138518,866.015421,848.666425,257.0,187.182878,2.828404,945.909912
Bamboi,1041.086726,365.0,20.836428,78.279627,814.206043,923.577379,365.0,112.407802,73.350016,984.128883,988.862450,365.0,208.882178,89.788584,1065.681059,953.311566,0.0,108.776828,0.000000,892.398340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yilou,942.434294,0.0,17.393241,0.000000,749.617800,887.995466,0.0,124.595555,0.000000,948.616621,781.458403,0.0,20.829566,0.000000,765.908095,831.269278,0.0,105.007485,0.000000,817.367669
Ziou,982.875028,365.0,49.806161,16.815507,835.251057,889.809882,359.0,135.561261,56.807634,953.586745,1039.563125,363.0,158.333998,44.856950,1051.821795,896.200688,274.0,251.968079,49.546870,1037.767736
Gbanlou-bineda_Kopingue,NaN,113.0,49.297652,79.439418,NaN,NaN,86.0,80.596186,45.378986,NaN,NaN,57.0,248.593523,36.949647,NaN,NaN,70.0,46.678426,44.664263,NaN
Kara-kpessidè,NaN,0.0,387.234325,0.000000,NaN,NaN,0.0,322.422266,0.000000,NaN,NaN,0.0,376.113823,0.000000,NaN,NaN,0.0,605.638228,0.000000,NaN


In [27]:
# Generate list
selector2 = widgets.Dropdown(
options=series_annual["ET"].T.index,
value=series_annual["ET"].T.index[0])

output2 = widgets.Output()

# Set layout and display
form_item_layout = widgets.Layout(
    display='flex',
    justify_content='space-between'
)
display(widgets.VBox([selector2, output2], layout=form_item_layout))

# Re-generate multiplot
selector2.observe(combinedplot, names='value')